In [27]:
import sys
import arff
from io import StringIO
from sklearn import svm
import numpy as np
import pandas as pd
import time
import json
import csv
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import re
import javalang
from pandasgui import show

from sklearn.model_selection import cross_validate
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import minmax_scale
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import Binarizer
from sklearn.calibration import calibration_curve
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier


def weka_tokenizer(doc):
    delimiters_regexp = re.compile("[ |\n|\f|\r|\t|.|,|;|:|'|\"|(|)|?|!]")
    return list(filter(None, delimiters_regexp.split(doc)))

idFlakiesProjects = ['redpipe', 'vertexium', 'javaCasClient', 'c2mon', 'vertx', 'excelastic', 'rxjava2', 'tyrus', 'esper', 'yawp', 'luwak', 'fluentLoggerJava', 'delightNashornSandbox', 'dbScheduler', 'one', 'sawmill', 'springCloudZuulRatelimit', 'timely', 'sos', 'openpojo', 'ociJavaSdk', 'aletheia', 'pippo', 'recast4j', 'noxy', 'springCloudAws', 'vertxMqtt', 'vertxRabbitmqClient', 'admiral', 'carbonApimgt', 'riptide', 'fastjson', 'dubbo', 'webcollector', 'doanduyhaiAchilles', 'elasticjoblite', 'disconf', 'hutool', 'oryx', 'querydsl', 'helios', 'retrofit', 'javaWebsocket', 'undertow', 'alien4cloud', 'cukes', 'hsac', 'googdDataCl', 'springDataBean', 'jhispster', 'marineApi', 'junitQuickcheck', 'nexus', 'springDataEnvers', 'springWs', 'aismessages', 'unix4j', 'wikidata', 'activiti', 'jackrabbit', 'struts', 'jfreechart', 'httpRequest', 'wildflymaven', 'nifiRegistry', 'arangoddb', 'dnsjava', 'as2lib', 'whois', 'dbean', 'searchHighlighter', 'wildflymavenplugin', 'balana', 'limfs', 'jodatime', 'otto', 'dropwizard']

msr4FlakinessProjects = ['Achilles', 'ambari', 'assertj-core', 'checkstyle', 'commons-exec', 'dropwizard', 'hadoop', 'handlebars', 'hbase', 'hector', 'httpcore', 'jackrabbit-oak', 'jimfs', 'logback', 'ninja', 'okhttp', 'oozie', 'orbit', 'oryx', 'spring-boot', 'alluxio', 'togglz', 'undertow', 'wro4j', 'zxing']

In [28]:
data = pd.read_csv('sampled.csv')

In [29]:
data = data.reset_index()

# removing smells classes exceptionCatchingThrowing and dependentTest becouse not has value
data = data.drop(columns=['commit', 'testClass', 'testMethod', 'testFilePath', 'productionFilePath', 'relativeTestFilePath', 'relativeProductionFilePath', 'tsTestClass', 'tsTestMethod', 'is_sampled', 'dependentTest', 'exceptionCatchingThrowing', 'smellsCount', 'assertionRoulette', 'conditionalTestLogic', 'constructorInitialization', 'defaultTest', 'dependentTest', 'duplicateAssert', 'eagerTest', 'emptyTest', 'exceptionCatchingThrowing', 'generalFixture', 'ignoredTest', 'lazyTest', 'magicNumberTest', 'mysteryGuest', 'printStatement', 'redundantAssertion', 'resourceOptimism', 'sensitiveEquality', 'sleepyTest', 'unknownTest', 'verboseTest', 'tokens_parser', 'strings_parser', 'string_type_parser', 'anotations_parser'], axis=1)

data = data.replace(np.nan, 0)
data = data.replace(True, 1)
data = data.replace(False, 0)

data = data.replace('flaky', 1)
data = data.replace('nonflaky', 0)

In [30]:
# filtering by dataset
allMsr4flakiness = data.loc[data.dataset == 'msr4flakiness']
before = len(allMsr4flakiness)

# filtering by projects
smells = allMsr4flakiness.loc[allMsr4flakiness.project.isin(msr4FlakinessProjects)]
after = len(smells)

smells = smells.reset_index()

print(before, after)

2800 2777


In [31]:
vectorizer = CountVectorizer(analyzer='word', max_features=1551, tokenizer=weka_tokenizer)

bowToken = vectorizer.fit_transform(smells['vocabulary'])

train_vocabulary = vectorizer.vocabulary_

vocabularyData = pd.DataFrame(bowToken.toarray(), columns=vectorizer.get_feature_names())

train = smells.join(vocabularyData, lsuffix='_')

len(train.columns)

1559

In [32]:
keywords_vocabulary = {'abstract', 'assert', 'boolean', 'break', 'byte', 'case', 'catch', 'char', 'class', 'continue', 'default', 'do', 'double', 'else', 'enum', 'exports', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'instanceof', 'int', 'interface', 'long', 'modules', 'native', 'new', 'package', 'private', 'protected', 'public', 'requires', 'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'throws', 'transient', 'try', 'void', 'volatile', 'while', 'true', 'null', 'false', 'const', 'goto'}

keywordVectorizer = CountVectorizer(analyzer='word', max_features=56, vocabulary=keywords_vocabulary)

bowKeywords = keywordVectorizer.fit_transform(smells['keywords_parser'])
keywordData = pd.DataFrame(bowKeywords.toarray(), columns=keywordVectorizer.get_feature_names())

#creating keywordcount column
keywordData["count"] = keywordData[keywordData > 0].count(axis=1)
keywordData = keywordData.add_suffix('_keyword')

train = train.join(keywordData, lsuffix='_keyword')

len(train.columns)

1616

In [33]:
train.columns

Index(['level_0', 'index_', 'project', 'loc', 'vocabulary', 'keywords_parser',
       'klass', 'dataset_', '$radius', '$side',
       ...
       'this_keyword', 'throw_keyword', 'throws_keyword', 'transient_keyword',
       'true_keyword', 'try_keyword', 'void_keyword', 'volatile_keyword',
       'while_keyword', 'count_keyword'],
      dtype='object', length=1616)

In [34]:
train_y = train['klass']
train = train.drop(['keywords_parser', 'project', 'klass', 'dataset', 'dataset_', 'vocabulary', 'level_0', 'index_', 'index'], axis=1)

X_train, x_test, y_train, y_test = train_test_split(train, train_y, test_size=0.2, random_state=1) #, random_state=0

In [35]:
classifiers_old = {
        'randomForest': RandomForestClassifier(random_state=1), 
        'decisionTree': DecisionTreeClassifier(min_samples_leaf=1),
        'naiveBayes': GaussianNB(),
        'smo': CalibratedClassifierCV(LinearSVC(fit_intercept=False, tol=0.001, C=1, dual=False, max_iter=100000), method='sigmoid'),
        'knn': KNeighborsClassifier(n_neighbors=1, metric='euclidean'),
        'logisticRegression': LogisticRegression(max_iter=1000),
        'perceptron': CalibratedClassifierCV(Perceptron()),
        'lda': LinearDiscriminantAnalysis(),
    }

classifierStatistics = pd.DataFrame(columns=['features', 'process', 'step', 'classifier', 'acc', 'precision', 'recall', 'f1', 'mcc', 'auc', 'VP', 'FN'])

In [36]:
trained_classifier_old = {}

for key, classifier in classifiers_old.items():

    classifier.fit(X_train, y_train)
    predict = classifier.predict(x_test)
    y_probs = classifier.predict_proba(x_test)[:,1]

    acc = classifier.score(x_test, y_test)
    f1 = f1_score(y_test, predict, average='weighted')  
    cr = classification_report(y_test, predict, output_dict=True)  
    mcc = matthews_corrcoef(y_test, predict)
    auc = roc_auc_score(y_test, y_probs)

    classStatistics = {
        'features': 'vocabulary',
        'process': 'traditional',
        'step': 'training',
        'classifier': key,
        'acc': acc,
        'precision': cr['weighted avg']['precision'],
        'recall': cr['weighted avg']['recall'],
        'f1': f1,
        'auc': auc,
        'mcc': mcc
    }

    classifierStatistics = classifierStatistics.append(classStatistics, ignore_index=True)
    trained_classifier_old[key] = classifier

In [37]:
classifierStatistics[classifierStatistics.process == 'traditional']

,features,process,step,classifier,acc,precision,recall,f1,mcc,auc,VP,FN
0,vocabulary,traditional,training,randomForest,0.971223,0.971579,0.971223,0.971199,0.942660,0.989448,NaN,NaN
1,vocabulary,traditional,training,decisionTree,0.937050,0.937596,0.937050,0.937078,0.874501,0.937490,NaN,NaN
2,vocabulary,traditional,training,naiveBayes,0.951439,0.951548,0.951439,0.951416,0.902767,0.950959,NaN,NaN
3,vocabulary,traditional,training,smo,0.967626,0.967975,0.967626,0.967599,0.935440,0.992352,NaN,NaN
4,vocabulary,traditional,training,knn,0.929856,0.930938,0.929856,0.929889,0.860675,0.930736,NaN,NaN
5,vocabulary,traditional,training,logisticRegression,0.967626,0.967975,0.967626,0.967599,0.935440,0.994192,NaN,NaN
6,vocabulary,traditional,training,perceptron,0.965827,0.965952,0.965827,0.965811,0.931625,0.991794,NaN,NaN
7,vocabulary,traditional,training,lda,0.866906,0.871221,0.866906,0.866858,0.738208,0.876601,NaN,NaN


In [38]:
#Getting information gain ranking

informationGain = dict(zip(train.columns, mutual_info_classif(train, train_y, discrete_features=True)))
sortedInformationGain = sorted(informationGain, key=informationGain.get, reverse=True)

train_xy = train.join(train_y)

In [39]:
i = 0
sortedInformationGainPosition = []

for r in sortedInformationGain:    

    infGain = {
        'position': i, 
        'token': r, 
        'information_gain': informationGain[r],
        'total_ocurences': len(train_xy[ train_xy[r] > 0 ] ), 
        'total_flaky_occurences': len(train_xy[ (train_xy[r] > 0)  & (train_xy['klass'] == 1) ]), 
        'total_nonflaky_occurences': len(train_xy[ (train_xy[r] > 0)  & (train_xy['klass'] == 0) ])
    }

    sortedInformationGainPosition.append(infGain)
    i += 1

infGainDataset = pd.DataFrame(sortedInformationGainPosition)

In [40]:
infGainDataset.head(20)

,position,token,information_gain,total_ocurences,total_flaky_occurences,total_nonflaky_occurences
0,0,loc,0.254457,2777,1377,1400
1,1,new_keyword,0.153507,1825,1195,630
2,2,services,0.136059,469,469,0
3,3,throws_keyword,0.134179,2252,1324,928
4,4,count_keyword,0.131304,2777,1377,1400
5,5,get,0.121002,854,727,127
6,6,job,0.109289,387,387,0
7,7,call,0.094981,390,380,10
8,8,xml,0.079836,382,362,20
9,9,getstatus,0.074845,336,325,11


In [41]:
#Inter- intra-project test

In [42]:
test = data.loc[data.dataset == 'idFlakies']
test = test.reset_index()

len(test.columns)

test.columns

Index(['level_0', 'index', 'project', 'loc', 'vocabulary', 'keywords_parser',
       'klass', 'dataset'],
      dtype='object')

In [43]:
# Here we are using the trained vocabulary
idFlakiesVectorizer = CountVectorizer(analyzer='word', max_features=1551, tokenizer=weka_tokenizer, vocabulary=train_vocabulary) 

idFlakiesBowToken = idFlakiesVectorizer.fit_transform(test['vocabulary'])

idFlakiesVocabularyData = pd.DataFrame(idFlakiesBowToken.toarray(), columns=idFlakiesVectorizer.get_feature_names())

test = test.join(idFlakiesVocabularyData, lsuffix='_')

len(test.columns)

test.columns

Index(['level_0', 'index_', 'project', 'loc', 'vocabulary', 'keywords_parser',
       'klass', 'dataset_', '$radius', '$side',
       ...
       'yarn', 'yes', 'yield', 'z', '{', '{}', '{}ms', '}', '}</instance>',
       '}</value>'],
      dtype='object', length=1559)

In [44]:
keywordIdFlakiesVectorizer = CountVectorizer(analyzer='word', max_features=56, vocabulary=keywords_vocabulary)

bowIdFlakiesKeywords = keywordIdFlakiesVectorizer.fit_transform(test['keywords_parser'])
keywordIdFlakiesData = pd.DataFrame(bowIdFlakiesKeywords.toarray(), columns=keywordIdFlakiesVectorizer.get_feature_names())


#creating keywordcount column
keywordIdFlakiesData["count"] = keywordIdFlakiesData[keywordIdFlakiesData > 0].count(axis=1)
keywordIdFlakiesData = keywordIdFlakiesData.add_suffix('_keyword')

test = test.join(keywordIdFlakiesData, lsuffix='_keyword')

len(test.columns)

1616

In [45]:
msr4flakiness = test.loc[test.project.isin(msr4FlakinessProjects)]
idflakies = test[test.project.isin(idFlakiesProjects)]

msr4flakiness_y = msr4flakiness['klass']
idflakies_y = idflakies['klass']

msr4flakiness = msr4flakiness.drop(['keywords_parser', 'project', 'klass', 'dataset', 'vocabulary', 'level_0', 'index_', 'dataset_', 'index'], axis=1)
idflakies = idflakies.drop(['keywords_parser', 'project', 'klass', 'dataset', 'vocabulary', 'level_0', 'index_', 'dataset_', 'index'], axis=1)

In [46]:
idflakies.columns

Index(['loc', '$radius', '$side', '${2}', '${coord', '${hadoop', '*', '+', '-',
       '--',
       ...
       'this_keyword', 'throw_keyword', 'throws_keyword', 'transient_keyword',
       'true_keyword', 'try_keyword', 'void_keyword', 'volatile_keyword',
       'while_keyword', 'count_keyword'],
      dtype='object', length=1607)

In [47]:
print('msr4flakiness samples', len(msr4flakiness), len(msr4flakiness_y))
print('idflakies samples', len(idflakies), len(idflakies_y))

msr4flakiness samples 35 35
idflakies samples 120 120


In [48]:
list(set(X_train.columns) - set(msr4flakiness.columns))

[]

In [49]:
for key, classifier in trained_classifier_old.items():
    predict_msr4flakiness = classifier.predict(msr4flakiness)
    msr4flakiness_acc = classifier.score(msr4flakiness, msr4flakiness_y)        
    cr_msr4flakiness = classification_report(msr4flakiness_y, predict_msr4flakiness, output_dict=True, zero_division=1)
    tn_msr4flakiness_, fp_msr4flakiness_, fn_msr4flakiness_, tp_msr4flakiness_ = confusion_matrix(msr4flakiness_y, predict_msr4flakiness, labels=[0,1]).ravel()

    classStatistics = {
        'features': 'vocabulay',
        'process': 'traditional',
        'step': 'testing-intra-projects',
        'classifier': key,
        'acc': msr4flakiness_acc,
        'recall': cr_msr4flakiness['1']['recall'],
        'VP': tp_msr4flakiness_,
        'FN': fn_msr4flakiness_
    }

    classifierStatistics = classifierStatistics.append(classStatistics, ignore_index=True)

    predict_idflakies = classifier.predict(idflakies)
    idflakies_acc = classifier.score(idflakies, idflakies_y)
    cr_idflakies = classification_report(idflakies_y, predict_idflakies, output_dict=True, zero_division=1)
    tn_idflakies_, fp_idflakies_, fn_idflakies_, tp_idflakies_ = confusion_matrix(idflakies_y, predict_idflakies, labels=[0, 1]).ravel()

    classStatistics = {
        'features': 'vocabulay',
        'process': 'traditional',
        'step': 'testing-inter-projects',
        'classifier': key,
        'acc': idflakies_acc,
        'recall': cr_idflakies['1']['recall'],
        'VP': tp_idflakies_,
        'FN': fn_idflakies_
    }

    classifierStatistics = classifierStatistics.append(classStatistics, ignore_index=True)

In [50]:
classifierStatistics[(classifierStatistics.process == 'traditional') & (classifierStatistics.step == 'testing-intra-projects')]

,features,process,step,classifier,acc,precision,recall,f1,mcc,auc,VP,FN
8,vocabulay,traditional,testing-intra-projects,randomForest,0.171429,NaN,0.171429,NaN,NaN,NaN,6.0,29.0
10,vocabulay,traditional,testing-intra-projects,decisionTree,0.314286,NaN,0.314286,NaN,NaN,NaN,11.0,24.0
12,vocabulay,traditional,testing-intra-projects,naiveBayes,0.171429,NaN,0.171429,NaN,NaN,NaN,6.0,29.0
14,vocabulay,traditional,testing-intra-projects,smo,0.085714,NaN,0.085714,NaN,NaN,NaN,3.0,32.0
16,vocabulay,traditional,testing-intra-projects,knn,0.571429,NaN,0.571429,NaN,NaN,NaN,20.0,15.0
18,vocabulay,traditional,testing-intra-projects,logisticRegression,0.200000,NaN,0.200000,NaN,NaN,NaN,7.0,28.0
20,vocabulay,traditional,testing-intra-projects,perceptron,0.342857,NaN,0.342857,NaN,NaN,NaN,12.0,23.0
22,vocabulay,traditional,testing-intra-projects,lda,0.285714,NaN,0.285714,NaN,NaN,NaN,10.0,25.0


In [51]:
classifierStatistics[(classifierStatistics.process == 'traditional') & (classifierStatistics.step == 'testing-inter-projects')]

,features,process,step,classifier,acc,precision,recall,f1,mcc,auc,VP,FN
9,vocabulay,traditional,testing-inter-projects,randomForest,0.266667,NaN,0.266667,NaN,NaN,NaN,32.0,88.0
11,vocabulay,traditional,testing-inter-projects,decisionTree,0.366667,NaN,0.366667,NaN,NaN,NaN,44.0,76.0
13,vocabulay,traditional,testing-inter-projects,naiveBayes,0.125000,NaN,0.125000,NaN,NaN,NaN,15.0,105.0
15,vocabulay,traditional,testing-inter-projects,smo,0.166667,NaN,0.166667,NaN,NaN,NaN,20.0,100.0
17,vocabulay,traditional,testing-inter-projects,knn,0.225000,NaN,0.225000,NaN,NaN,NaN,27.0,93.0
19,vocabulay,traditional,testing-inter-projects,logisticRegression,0.300000,NaN,0.300000,NaN,NaN,NaN,36.0,84.0
21,vocabulay,traditional,testing-inter-projects,perceptron,0.333333,NaN,0.333333,NaN,NaN,NaN,40.0,80.0
23,vocabulay,traditional,testing-inter-projects,lda,0.558333,NaN,0.558333,NaN,NaN,NaN,67.0,53.0
